In [1]:
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
import pandas as pd
import time
from datetime import datetime

In [2]:
df=pd.read_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/Earthquake.csv')
df

,Lat,Lon,Depth,Mag,An,Previous
0,57.090,-153.480,0.0,8.3,1900,0
1,60.000,-135.000,0.0,7.1,1901,0
2,52.000,-160.000,0.0,6.6,1901,0
3,51.450,-171.020,0.0,7.8,1901,0
4,52.380,-167.450,0.0,7.8,1902,0
...,...,...,...,...,...,...
363065,16.625,-98.755,35.0,4.0,2012,0
363066,52.788,171.431,48.4,4.5,2012,0
363067,14.083,-92.184,62.4,4.4,2012,0
363068,-61.596,154.288,10.0,5.2,2012,0


In [3]:
from pandas import read_csv
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
# load data


array = df.values
X = array[:,0:4]
Y = array[:, 5]
# feature extraction
test = SelectKBest(score_func=f_classif, k= 'all')
fit = test.fit(X, Y)
# summarize scores
set_printoptions(precision=4)
print(fit.scores_)
features = fit.transform(X)
# summarize selected features
print(features[0:4,:])

[106.0991  75.7125  20.6023 167.8326]
[[  57.09 -153.48    0.      8.3 ]
 [  60.   -135.      0.      7.1 ]
 [  52.   -160.      0.      6.6 ]
 [  51.45 -171.02    0.      7.8 ]]


In [7]:
df.to_csv('Global.csv')

In [13]:
col1 = df[['Previous','Lat', 'Lon', 'Depth']]
col2 = df['Mag']
#Convert to Numpy array
InputX1 = col1.to_numpy()
InputY1 = col2.to_numpy()
print(InputX1)

[[   0.      57.09  -153.48     0.   ]
 [   0.      60.    -135.       0.   ]
 [   0.      52.    -160.       0.   ]
 ...
 [   0.      14.083  -92.184   62.4  ]
 [   0.     -61.596  154.288   10.   ]
 [   0.      -6.95   154.485   52.8  ]]


In [5]:
X1_min = np.amin(InputX1,0)     
X1_max = np.amax(InputX1,0)   
print("Mininum values:",X1_min)
print("Maximum values:",X1_max)
Y1_min = np.amin(InputY1)     
Y1_max = np.amax(InputY1) 
InputX1_norm = (InputX1-X1_min)/(X1_max-X1_min)
InputY1_norm = InputY1  #No normalization in output

#Reshape
Xfeatures = 4 #Number of input features
Yfeatures = 1 #Number of input features
samples = 363070 # Number of samples

InputX1_reshape = np.resize(InputX1_norm,(samples,Xfeatures))
InputY1_reshape = np.resize(InputY1_norm,(samples,Yfeatures))

Mininum values: [0. 0.]
Maximum values: [  4.  735.8]


In [6]:
batch_size = 2000
InputX1train = InputX1_reshape[0:batch_size,:]
InputY1train = InputY1_reshape[0:batch_size,:]
#Validation data
v_size = 2500
InputX1v = InputX1_reshape[batch_size:batch_size+v_size,:]
InputY1v = InputY1_reshape[batch_size:batch_size+v_size,:]

In [7]:
learning_rate = 0.001
training_iterations = 1000
display_iterations = 200

In [8]:
tf.compat.v1.disable_eager_execution()
#Input
X = tf.compat.v1.placeholder(tf.float32,shape=(None,Xfeatures))
#tf.compat.v1.placeholder(shape=[None, 2], dtype=tf.float32)
#Output
Y = tf.compat.v1.placeholder(tf.float32)


In [9]:
#Neurons
L1 = 3
L2 = 3
L3 = 3

#Layer1 weights
W_fc1 = tf.Variable(tf.random_uniform([Xfeatures,L1]))
b_fc1 = tf.Variable(tf.constant(0.1,shape=[L1]))

#Layer2 weights
W_fc2 = tf.Variable(tf.random_uniform([L1,L2]))
b_fc2 = tf.Variable(tf.constant(0.1,shape=[L2]))

#Layer3 weights
W_fc3 = tf.Variable(tf.random_uniform([L2,L3]))
b_fc3 = tf.Variable(tf.constant(0.1,shape=[L3]))

#Output layer weights
W_fO= tf.Variable(tf.random_uniform([L3,Yfeatures]))
b_fO = tf.Variable(tf.constant(0.1,shape=[Yfeatures]))

In [10]:
#Layer 1
matmul_fc1=tf.matmul(X, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(matmul_fc1)   #ReLU activation
#Layer 2
matmul_fc2=tf.matmul(h_fc1, W_fc2) + b_fc2
h_fc2 = tf.nn.relu(matmul_fc2)   #ReLU activation
#Layer 3
matmul_fc3=tf.matmul(h_fc2, W_fc3) + b_fc3
h_fc3 = tf.nn.relu(matmul_fc3)   #ReLU activation
#Output layer
matmul_fc4=tf.matmul(h_fc3, W_fO) + b_fO
output_layer = matmul_fc4  #linear activation

In [11]:
#Loss function
mean_square =  tf.reduce_mean(tf.square(Y-output_layer))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_square)

#Operation to save variables
saver = tf.train.Saver()

In [12]:
#Initialization and session
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print("Training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    for i in range(training_iterations):
        sess.run([train_step],feed_dict={X:InputX1train,Y:InputY1train})
        if i%display_iterations ==0:
            print("Training loss is:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}),"at itertion:",i)
            print("Validation loss is:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}),"at itertion:",i)
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/earthquake_model.ckpt")
    print("Model saved in file: %s" % save_path)

    print("Final training loss:",sess.run([mean_square],feed_dict={X:InputX1train,Y:InputY1train}))
    print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))

Training loss: [10.627669]
Training loss is: [10.518108] at itertion: 0
Validation loss is: [9.021625] at itertion: 0
Training loss is: [2.145586] at itertion: 200
Validation loss is: [1.4782654] at itertion: 200
Training loss is: [1.21938] at itertion: 400
Validation loss is: [0.77178305] at itertion: 400
Training loss is: [0.9268791] at itertion: 600
Validation loss is: [0.4828215] at itertion: 600
Training loss is: [0.8183984] at itertion: 800
Validation loss is: [0.36056963] at itertion: 800
Model saved in file: /tmp/earthquake_model.ckpt
Final training loss: [0.77690756]
Final validation loss: [0.3126057]


In [17]:
#Testing
lat = input("Enter Latitude between -90 to 90:")
long = input("Enter Longitude between -180 to 180:")
depth = input("Enter Depth between 0 to 800:")
days = input("Enter the number of days:")
InputX2 = np.asarray([[lat,long,depth,days]],dtype=np.float32)
InputX2_norm = (InputX2-X1_min)/(X1_max-X1_min)
InputX1test = np.resize(InputX2_norm,(1,Xfeatures))
with tf.Session() as sess:
    # Restore variables from disk for validation.
    saver.restore(sess, "/tmp/earthquake_model.ckpt")
    print("Model restored.")
    #print("Final validation loss:",sess.run([mean_square],feed_dict={X:InputX1v,Y:InputY1v}))
    print("output:",sess.run([output_layer],feed_dict={X:InputX1test}))

Enter Latitude between -90 to 90:10
Enter Longitude between -180 to 180:100
Enter Depth between 0 to 800:100
Enter the number of days:300
INFO:tensorflow:Restoring parameters from /tmp/earthquake_model.ckpt
Model restored.
output: [array([[3.1074939]], dtype=float32)]
